<a href="https://colab.research.google.com/github/antonutellaa/pete-comp-meth-public/blob/main/python_exercises6_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's begin by loading the `plotnine` module; call it `p9` for convenience.

In [5]:
import plotnine as p9

We'll also need `pandas` (call it `pd`):

In [7]:
import pandas as pd

We'll want to connect to the shared Google Drive to get access to our data. Remember, this is always done the same way:

In [8]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Now load the LABEVENTS.csv and PATIENTS.csv data from MIMIC:

In [9]:
labevents = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/LABEVENTS.csv")
patients = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/PATIENTS.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/LABEVENTS.csv'

Take a look at both datasets to get a sense of how it is structured:

In [ ]:
labevents

In [ ]:
patients

In the `labevents` dataframe, the `charttime` column represents date and time. Use `pd.to_datetime` to tell the computer this:

In [ ]:
labevents['charttime'] = pd.to_datetime(labevents['charttime'])

### Bar chart

Let's start simply: Make a bar chart showing the `gender` data from `patients`. You should end up with two bars: one for `M` one for `F`. Which gender is more common in our dataset?

In [ ]:
p9.ggplot(patients) + p9.geom_bar(p9.aes(x="gender"))

### Time series

Select blood potassium concentration in mEq/L (`itemid == 50971`) for patient `40310`.


In [ ]:
potassium = labevents[(labevents['itemid']==50971) & (labevents['subject_id']==40310)]

Make a line graph of the measurements (`valuenum`) vs date (`charttime`). Note that ggplot automatically handles timestamps; we do not need to do anything special for dates.

In [ ]:
p9.ggplot(potassium) + p9.geom_line(p9.aes(x="charttime",y="valuenum",group="subject_id"))

Did you give your graph meaningful x- and y-axis labels? If not, redo it but with labels "Date" for the x-axis and "[K] (mEq/L)" for the y-axis (you'll want to use `p9.labs`).

In [ ]:
p9.ggplot(potassium) + p9.geom_line(p9.aes(x="charttime",y="valuenum",group="subject_id")) + p9.labs(x="Date",y="[K] (mEq/L)")

How do you interpret the graph between mid-November and late December? Are there any time periods that stand out as not being like the rest?

**PUT YOUR ANSWER HERE**

### Box plot

So far we've seen how one particular person's blood potassium concentration changed over time.

Let's instead try to get a sense of the variability of blood sodium concentrations (`labevents['itemid'] == 50983`) within individuals but also how individuals compare. To do this, we'll use a boxplot (`geom_boxplot`). Specifically, make a boxplot that `group`s by `subject_id`, plots the concentrations (`valuenum`) along the y-axis, and positions the box across the x-axis beased on `subject_id`. To keep the graph readable, only include data for `subject_id < 20000`. Store the selected data in the variable `sodium` as we'll need it later.

In [ ]:
sodium = labevents[(labevents['itemid'] == 50983) & (labevents['subject_id'] < 20000)]

In [ ]:
p9.ggplot(sodium, p9.aes(x="subject_id", y="valuenum", group='subject_id')) + p9.geom_boxplot() + p9.theme(axis_text_x=p9.element_text(angle=90))

Is our visualization telling the right story?

Does the x-axis location mean anything? The `subject_id` only exists to distinguish unique patients; its numeric value has no meaning.

To get a better picture: make a new variable `sodium2` based on `sodium` (set it equal to `pd.DataFrame(sodium)`), then set the `subject_id` column to represent categorical values (`astype('category')`). Now regraph.

How does your graph compare to the previous one?

In [ ]:
sodium2 = pd.DataFrame(sodium)
sodium2['subject_id'] = sodium2['subject_id'].astype('category')
p9.ggplot(sodium2, p9.aes(x="subject_id", y="valuenum")) + p9.geom_boxplot() + p9.theme(axis_text_x=p9.element_text(angle=90))

Did you see any patients whose blood sodium levels seem unusual relative to the rest? (e.g. unusually high, unusually low, or unusually variable).

- high mean: 10036, 10067
- high variability: 10098, 100104


### Scatter plot

Let's load the population data from the Pima Indian diabetes dataset:




In [ ]:
diabetes = pd.read_csv('/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/diabetes.csv')

As always, start by looking at the data to get a sense of what it contains:

In [ ]:
diabetes

Make a scatter plot showing the relationship of `Glucose` and `Insulin` in the `diabetes` data.


In [ ]:
p9.ggplot(diabetes,p9.aes(x="Glucose",y="Insulin")) + p9.geom_point()

You can see that there are a lot of times where insulin has a zero value. Let's filter those out and continue.

In [9]:
diabetes_insulin = diabetes[diabetes['Insulin']!=0]

There are so many points, it's hard to see where the most likely values are in the basic scatter plot. To get a better sense, add a `p9.geom_density_2d` to overlay a contour map of the point density. Specify a `color` so that it stands out:

In [ ]:
p9.ggplot(diabetes_insulin,p9.aes(x="Glucose",y="Insulin")) + p9.geom_point() + p9.geom_density_2d(color='red')

Finally, let's add a smoothed curve `p9.geom_smooth` showing the overall trend. Again, specify a color so that you can see the trend curve:

In [ ]:
p9.ggplot(diabetes_insulin,p9.aes(x="Glucose",y="Insulin")) + p9.geom_point() + p9.geom_density_2d(color='red') + p9.geom_smooth(method='lm',color='blue',se=False)